In [1]:
from netCDF4 import Dataset  # http://code.google.com/p/netcdf4-python/
import os
####################you will need to change some paths here!#####################
#list of input files
dir_in='f:/data/tc_wakes/database/info/'
dir_out='f:/data/tc_wakes/database/sst/'
#output files
#filename_out_nc='F:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/data_so_far.nc'
#filename_out_kml='F:/data/cruise_data/saildrone/baja-2018/daily_files/sd-1002/data_so_far.kml'
#################################################################################
import datetime as dt
import xarray as xr
from datetime import datetime
import pandas
import matplotlib as mpl
#import openpyxl
#from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import math
#from math import cos, radians

In [2]:
def ncdump(nc_fid, verb=True):
    '''
    ncdump outputs dimensions, variables and their attribute information.
    The information is similar to that of NCAR's ncdump utility.
    ncdump requires a valid instance of Dataset.
    Parameters
    ----------
    nc_fid : netCDF4.Dataset
        A netCDF4 dateset object
    verb : Boolean
        whether or not nc_attrs, nc_dims, and nc_vars are printed
    Returns
    -------
    nc_attrs : list
        A Python list of the NetCDF file global attributes
    nc_dims : list
        A Python list of the NetCDF file dimensions
    nc_vars : list
        A Python list of the NetCDF file variables
    '''

    def print_ncattr(key):
        """
        Prints the NetCDF file attributes for a given key
        Parameters
        ----------
        key : unicode
            a valid netCDF4.Dataset.variables key
        """
        try:
            print("\t\ttype:", repr(nc_fid.variables[key].dtype))
            for ncattr in nc_fid.variables[key].ncattrs():
                print('\t\t%s:' % ncattr, \
                      repr(nc_fid.variables[key].getncattr(ncattr)))
        except KeyError:
            print("\t\tWARNING: %s does not contain variable attributes" % key)

    # NetCDF global attributes
    nc_attrs = nc_fid.ncattrs()
    if verb:
        print("NetCDF Global Attributes:")
        for nc_attr in nc_attrs:
            print('\t%s:' % nc_attr, repr(nc_fid.getncattr(nc_attr)))
    nc_dims = [dim for dim in nc_fid.dimensions]  # list of nc dimensions
    # Dimension shape information.
    if verb:
        print("NetCDF dimension information:")
        for dim in nc_dims:
            print("\tName:", dim)
            print("\t\tsize:", len(nc_fid.dimensions[dim]))
            print_ncattr(dim)
    # Variable information.
    nc_vars = [var for var in nc_fid.variables]  # list of nc variables
    if verb:
        print("NetCDF variable information:")
        for var in nc_vars:
            if var not in nc_dims:
                print('\tName:', var)
                print("\t\tdimensions:", nc_fid.variables[var].dimensions)
                print("\t\tsize:", nc_fid.variables[var].size)
                print_ncattr(var)
    return nc_attrs, nc_dims, nc_vars

In [9]:
for root, dirs, files in os.walk(dir_in, topdown=False):
    for name in files:
        fname_in=os.path.join(root, name)
        print(fname_in[31:39])


2004\001
2004\001
2004\002
2004\003
2004\004
2004\005
2004\006
2004\007
2004\008
2004\009
2004\011
2004\012
2004\013
2004\014
2004\015
2004\016
2004\017
2004\018
2004\019
2004\021
2004\022
2004\023
2004\024
2004\025
2004\026
2004\028
2004\029
2004\030
2004\031
2004\032
2004\033
2004\034
2004\037
2004\038
2004\039
2004\040
2004\041
2004\043
2004\044
2004\045
2004\046
2004\047
2004\048
2004\049
2004\051
2004\052
2004\053
2004\054
2004\055
2004\058
2004\059
2004\060
2004\062
2004\063
2004\064
2004\065
2004\066
2004\068
2004\069
2004\071
2004\072
2004\073
2004\074
2004\075
2004\076
2004\081
2004\084
2004\087
2004\089
2004\091
2004\092
2004\093
2005\002
2005\003
2005\004
2005\005
2005\006
2005\007
2005\009
2005\010
2005\011
2005\012
2005\014
2005\015
2005\016
2005\017
2005\018
2005\019
2005\020
2005\021
2005\022
2005\023
2005\024
2005\025
2005\026
2005\027
2005\028
2005\029
2005\031
2005\034
2005\035
2005\038
2005\042
2005\043
2005\044
2005\045
2005\047
2005\048
2005\049
2005\053
2005\055
2

In [73]:
###### dir_mur = 'F:/data/sst/jpl_mur/v4.1/'
date_1858 = dt.datetime(1858,11,17,0,0,0) # start date is 11/17/1958
for root, dirs, files in os.walk(dir_in, topdown=False):
    for name in files:
        fname_in=os.path.join(root, name)
        print(name,fname_in)
        dsx = xr.open_dataset(fname_in)
        lats = dsx.lat
        lons = dsx.lon
        dysince = dsx.time   
        minlon=min(lons[0,:].values)-10
        maxlon=max(lons[0,:].values)+10
        minlat=min(lats[0,:].values)-10
        maxlat=max(lats[0,:].values)+10
        dx=360./36000.
        dy=180./17999.
        ix1=int(round((minlon+(180.-dx))/dx))
        ix2=int(round((maxlon+(180.-dx))/dx))
        iy1=int(round((minlat+(90.-dy))/dy))
        iy2=int(round((maxlat+(90.-dy))/dy))
        print(minlon,maxlon,minlat,maxlat)
        dims=lats.shape
        print(dims[1],ix1,ix2,iy1,iy2)
        tem_date=[0]*dims[1] #print(dysince.values)
        tem_jdy=[0]*dims[1] #print(dysince.values)
        tem_hrs=[0]*dims[1] #print(dysince.values)
        xdim=ix2-ix1
        ydim=iy2-iy1
        tdim=dims[1]
        if xdim>10000:
            ix1=int(round((minlon+(180.-dx))/dx))+36000  #this is when storm wraps from 180 to -180, ix1 > 36000
            ix2=int(round((maxlon+(180.-dx))/dx))
            xdim=abs(xdim-36000)           
        sst_sv= np.zeros([tdim,ydim,xdim], dtype="float")
        print('sst_sv',sst_sv.shape)
        for i in range(0,dims[1]):
            tem_hrs[i] = (dysince[0,i].values - math.floor(dysince[0,i].values))*24.
            tem_dy = math.floor(dysince[0,i].values)
            tem_date[i]=date_1858+dt.timedelta(days=float(tem_dy))  #create new time array that can be queried for year etc
            tem_jdy[i]=tem_date[i].timetuple().tm_yday  #create new time array that can be queried for year etc
            syr=str(tem_date[i].year)
            smon=str(tem_date[i].month)
            sdym=str(tem_date[i].day)
            sjdy=str(tem_date[i].timetuple().tm_yday)
            fname_tem=syr + smon.zfill(2) + sdym.zfill(2) + '090000-JPL-L4_GHRSST-SSTfnd-MUR-GLOB-v02.0-fv04.1.nc'
            mur_filename = dir_mur + syr + '/' + sjdy.zfill(3) + '/' + fname_tem
            clim_filename='F:/data/sst/jpl_mur/v4.1/clim/clim2_' + sjdy.zfill(3) +'_2003_2013_MUR-GLOB-v02.0-fv04.1.nc'
            nc_fid = Dataset(mur_filename, 'r')
            nc_fid2 = Dataset(clim_filename, 'r')
            #sst_clim = nc_fid2.variables['sst']
            #print(sst_clim.shape)
            if i==0:
                print(iy1,iy2,ix1,ix2,iy2-iy1,ix2-ix1)
            if ix1<=36000 and ix2<=36000:
                sst = nc_fid.variables['analysed_sst'][0,iy1:iy2,ix1:ix2]-273.15
                sst_clim = nc_fid2.variables['sst'][iy1:iy2,ix1:ix2]
            if ix1>36000 and ix2>36000:
                sst = nc_fid.variables['analysed_sst'][0,iy1:iy2,(ix1-36000):(ix2-36000)]-273.15
                sst_clim = nc_fid2.variables['sst'][iy1:iy2,(ix1-36000):(ix2-36000)]
            if ix1<=36000 and ix2>36000:
                sst1 = nc_fid.variables['analysed_sst'][0,iy1:iy2,ix1:]-273.15
                sst2 = nc_fid.variables['analysed_sst'][0,iy1:iy2,:ix2-36000]-273.15
                sst = np.append(sst1,sst2, axis=1)
                sst1 = nc_fid2.variables['sst'][iy1:iy2,ix1:]
                sst2 = nc_fid2.variables['sst'][iy1:iy2,:ix2-36000]
                sst_clim = np.append(sst1,sst2, axis=1)
            if ix1>36000 and ix2<36000:
                sst1 = nc_fid.variables['analysed_sst'][0,iy1:iy2,ix2:]-273.15
                sst2 = nc_fid.variables['analysed_sst'][0,iy1:iy2,:ix1-36000]-273.15
                sst = np.append(sst1,sst2, axis=1)
                sst1 = nc_fid2.variables['sst'][iy1:iy2,ix2:]
                sst2 = nc_fid2.variables['sst'][iy1:iy2,:ix1-36000]
                sst_clim = np.append(sst1,sst2, axis=1)
            nc_fid.close()
            nc_fid2.close()
            if i==0:
                print('sst',sst.shape,'sst_sv',sst_sv.shape)
            sst_sv[i,:,:]=sst-sst_clim

        ilen=len(fname_in)
        
        fname_out=dir_out + fname_in[31:39] + '_sst.nc'
        f = Dataset(fname_out,'w', format='NETCDF4') 
        tempgrp = f.createGroup('sst_data')
        tempgrp.createDimension('t', tdim)
        tempgrp.createDimension('y', ydim)
        tempgrp.createDimension('x', xdim)
        print(tdim,ydim,xdim)
        sst_netcdf = tempgrp.createVariable('sst', 'f4', ('t', 'y', 'x'))
        sst_netcdf[:] = sst_sv
        f.close()
        


001annual_storm_info.nc f:/data/tc_wakes/database/info/2004\001annual_storm_info.nc
53.299998585134745 77.3999984934926 -16.999999843537807 5.8000000938773155
26 23329 25739 7299 9578
sst_sv (26, 2279, 2410)
7299 9578 23329 25739 2279 2410
sst (2279, 2410) sst_sv (26, 2279, 2410)
26 2279 2410
002annual_storm_info.nc f:/data/tc_wakes/database/info/2004\002annual_storm_info.nc
41.8999988399446 89.09999823197722 -26.49999963119626 3.6000001430511475
63 22189 26909 6349 9358
sst_sv (63, 3009, 4720)
6349 9358 22189 26909 3009 4720
sst (3009, 4720) sst_sv (63, 3009, 4720)
63 3009 4720
003annual_storm_info.nc f:/data/tc_wakes/database/info/2004\003annual_storm_info.nc
28.99999912828207 75.19999854266644 -50.299999099224806 0.4000002145767212
72 20899 25519 3969 9038
sst_sv (72, 5069, 4620)
3969 9038 20899 25519 5069 4620
sst (5069, 4620) sst_sv (72, 5069, 4620)
72 5069 4620
004annual_storm_info.nc f:/data/tc_wakes/database/info/2004\004annual_storm_info.nc
82.99999792128801 109.39999777823687

MemoryError: 

In [46]:
print(sst.shape,sst_sv.shape)
print(ix1,ix2,iy1,iy2)
print(sst.shape)
print(sst1.shape)
print(sst2.shape)
#sst = np.append(sst1,sst2, axis=1)
print(sst.shape)

(4010, 2480) (64, 3290, 4900)
31099 35999 18638 21928
(4010, 2480)
(4010, 1231)
(4010, 1249)
(4010, 2480)


In [66]:
minlon=-160.0
maxlon=170.0
ix1=int(round((minlon+(180.-dx))/dx))
ix2=int(round((maxlon+(180.-dx))/dx))
xdim=ix2-ix1
print(ix1,ix2,xdim)
if xdim>10000:
    ix1=int(round((minlon+(180.-dx))/dx))+36000
    ix2=int(round((maxlon+(180.-dx))/dx))
    xdim=abs(xdim-36000)    
print(ix1,ix2,xdim)

1999 34999 33000
37999 34999 3000


In [38]:
#fname_out=dir_out + fname_in[31:39] + '_sst2.nc'
#foo = xr.DataArray(sst_sv);
#foo.to_netcdf(fname_out)

In [39]:
print(dir_out)
print(fname_out)

f:/data/tc_wakes/database/sst/
f:/data/tc_wakes/database/sst/2004\003_sst2.nc


In [69]:
filename_clim='F:/data/sst/jpl_mur/v4.1/clim/clim2_' + sjdy +'_2003_2013_MUR-GLOB-v02.0-fv04.1.nc'

In [70]:
print(filename_clim)

F:/data/sst/jpl_mur/v4.1/clim/clim2_273_2003_2013_MUR-GLOB-v02.0-fv04.1.nc
